##  CNN model for fashion mnist by Zalando in Azure Machine Learning
based on
https://github.com/aysemutlu/AzureML-FashionMNIST/blob/master/Fashion%20MNIST%20Image%20Classification%20-%20Azure%20ML%20SDK%20Training.py

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-keras

https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py


## Checking installed packages

In [2]:
import subprocess
import sys
reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

In [3]:
#  azureml-sdk[notebooks]
if  not 'azureml-sdk' in installed_packages:
  !pip install azureml-sdk[notebooks]

In [67]:
#  azureml-opendatasets
if  not 'azureml-opendatasets' in installed_packages:
  !pip install azureml-opendatasets

In [5]:
#  matplotlib
if  not 'matplotlib' in installed_packages:
  !pip install matplotlib

In [70]:
#!ls -la data/

In [71]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.72


## Interacting with Azure Machine Learning

In [7]:
!az login --output none

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FNEXA6JMS to authenticate.


In [ ]:
!az account list -o table

## Connect To Workspace

Initialize a [Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the prerequisites step. Workspace.from_config() creates a workspace object from the details stored in config.json.

In [ ]:
from azureml.core import Workspace
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [78]:
# our experiment name
experiment_name = 'deeplearning_mnist_fashion'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)
print(exp)

Experiment(Name: deeplearning_mnist_fashion,
Workspace: my-ml)


## Creating Virtual Machines for computing

### Create Compute Target

A [compute target](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py) is a designated compute resource/environment where you run your training script or host your service deployment. This location may be your local machine or a cloud-based compute resource. Compute targets can be reused across the workspace for different runs and experiments. 

For this tutorial, we will create an auto-scaling [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute?view=azure-ml-py) cluster, which is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. To create the cluster, we need to specify the following parameters:

- `vm_size`: The is the type of GPUs that we want to use in our cluster. For this tutorial, we will use **STANDARD_D2_V2** .
- `idle_seconds_before_scaledown`: This is the number of seconds before a node will scale down in our auto-scaling cluster. We will set this to **3600** seconds. 
- `min_nodes`: This is the minimum numbers of nodes that the cluster will have. To avoid paying for compute while they are not being used, we will set this to **0** nodes.
- `max_modes`: This is the maximum number of nodes that the cluster will scale up to. Will will set this to **1** nodes.

**When jobs are submitted to the cluster it takes approximately 5 minutes to allocate new nodes** 

In [79]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

cluster_name='cpumlcluster'
# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", cluster_name)
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 1)

# This example uses CPU VM. For using CPU VM, set SKU to STANDARD_D2_V2 for GPU set STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target. just use it. ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                idle_seconds_before_scaledown=3600,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

Found compute target. just use it. cpumlcluster


In [80]:
import os
script_folder = './mnist-fashion'
os.makedirs(script_folder, exist_ok=True)

## Register Datastore
A [Datastore](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py) is used to store connection information to a central data storage. This allows you to access your storage without having to hard code this (potentially confidential) information into your scripts. 

In this tutorial, the model was been previously prepped and uploaded into a central [Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) container. We will register this container into our workspace as a datastore using a [shared access signature (SAS) token](https://docs.microsoft.com/en-us/azure/storage/common/storage-sas-overview). 



We need to define the following parameters to register a datastore:

- `ws`: The workspace object
- `datastore_name`: The name of the datastore, case insensitive, can only contain alphanumeric characters and _.
- `container_name`: The name of the azure blob container.
- `account_name`: The storage account name.
- `sas_token`: An account SAS token, defaults to None.

#### If the datastore has already been registered, then you (and other users in your workspace) can directly run this cell.

In [145]:
ds = ws.get_default_datastore()
print('ds.datastore_type:',ds.datastore_type, 'ds.account_name:',ds.account_name,'ds.container_name:',ds.container_name,sep = '\t')

ds.datastore_type:	AzureBlob	ds.account_name:	myml2814190254	ds.container_name:	azureml-blobstore-91798245-c368-4923-b448-4de054f267a7


In [81]:
# On first use uncomment the line below to upload your data
ds.upload(src_dir='./data', target_path='fashiondata', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/t10k-images-idx3-ubyte.gz
Uploading ./data/t10k-labels-idx1-ubyte.gz
Uploading ./data/train-images-idx3-ubyte.gz
Uploading ./data/train-labels-idx1-ubyte.gz
Uploaded ./data/t10k-labels-idx1-ubyte.gz, 1 files out of an estimated total of 4
Uploaded ./data/train-labels-idx1-ubyte.gz, 2 files out of an estimated total of 4
Uploaded ./data/t10k-images-idx3-ubyte.gz, 3 files out of an estimated total of 4
Uploaded ./data/train-images-idx3-ubyte.gz, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_bf56fa64a95b4007b23872a83887828f

### Preparing python script for building model ->  train.py file

In [130]:
%%writefile $script_folder/train.py

import tensorflow as tf
import os
import time
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
print("tensorflow Version is: " + str(tf.__version__))

import numpy as np
os.environ['KERAS_BACKEND'] = 'tensorflow'
#from keras import backend as K
#from tensorflow.keras import backend as K
from tensorflow.python.keras import backend as K
print(os.environ['KERAS_BACKEND'])

# * Import all the Keras functions we will need to use to create a Convolutional Neural Network (CNN)

#Fashion MNIST Dataset CNN model development: https://github.com/zalandoresearch/fashion-mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import utils, losses, optimizers
import matplotlib.pyplot as plt

from azureml.core import Run

# start an Azure ML run
run = Run.get_context()


parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--batch-size', type=int, dest='batch_size', default=128, help='mini batch size for training')
parser.add_argument('--first-layer-neurons', type=int, dest='n_hidden_1', default=100,
                    help='# of neurons in the first layer')
parser.add_argument('--second-layer-neurons', type=int, dest='n_hidden_2', default=100,
                    help='# of neurons in the second layer')
parser.add_argument('--epochs', type=int, dest='epochs', default=5,
                    help='# number of epochs')
parser.add_argument('--learning-rate', type=float, dest='learning_rate', default=0.001, help='learning rate')

args = parser.parse_args()

data_folder = args.data_folder

learning_rate = args.learning_rate

print('training dataset is stored here:', data_folder)

# * We setup some variables for example how many classes there are [0-9] as well as batch size to send the training sample of data in to the model and epochs is how many iterations/run thoroughs of the data there are
# * Each image is of size 28 x 28 pixels

#no. of classes
num_classes = 10
run.log('Number of classes',num_classes)
# batch size and training iterations (epochs)
# batch_size = 128
batch_size = args.batch_size
#epochs = 3 # was 24
epochs = args.epochs

#input image dimensions
img_rows,img_cols = 28,28


# * In this section lets have a look at the data
# * we pull in the fashion MNIST data from the Keras library into training and testing sets
# * X stands for features and y stands for labels
# * From the shape statements in the output you can see there are 60,000! training images and 10,000 test images - so a lot more data to use in this model
# * We then show one of the images from the training set and the corresponding text label for the image
# 
# > change the img_index field to any number between 0 - 60000 to see different images


#data for train and testing
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape, 'train set')
print(x_test.shape, 'test set')

# Define the text labels
fashion_mnist_labels = ["Top",          # index 0
                        "Trouser",      # index 1
                        "Jumper",       # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Trainer",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

img_index=90
label_index = y_train[img_index]
plt.imshow(x_train[img_index])
print('Label Index: ' + str(label_index) + " Fashion Labels: " + (fashion_mnist_labels[label_index]))
run.log_image('Sample image', plot=plt)

# * In this section we normalise the data so the pixel values in the image are between 0 - 1 instead of 0 - 255 pixel values. This will help the model to converge and the math becomes easier with smaller numbers
# * We also [one-hot-encode](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science#) the labels in matrices with 0's and 1's in them only. We do this so the model does not deem any category 0-9 in a numeric ranking. For example it won't think that tshirts[0] always come before trousers[1] when actually these are IDs of the classes not something to be evaluated
# * finally as we are deadling with greyscale images we have a depth number = 1 that might be interpreted different dpending on the framework used (CNTK, Tensorflow etc)


#type convert and scale the test and training data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#one-hot encoding
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test,  num_classes)

#formatting issues for depth of image (greyscale = 1) with different kernels (tensorflow, cntk, etc)
if K.image_data_format()== 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0],1,img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
    x_test = x_test.reshape(x_test.shape[0],img_rows, img_cols,1)
    input_shape = (img_rows, img_cols,1)


# Now we are able to define the Convolutional Neural Network (CNN) in layers
# 
# ![CNN](./images/cnn.JPG "CNN")
# 
# * This is a **sequential model** meaning every layer passes information forward to the next layer of the network
# * **1st Convoltuional Layer** - extracts features from data source, these are kernels/filters and feature maps. Feature maps passed to the  next layer. This layer also has a ReLu activation function - Y = max(0, x) this removes any value <0 and prevents vanishing gradients or weights <0
# * **2nd pooling layer ** - reduces dimensionality, reduce compute and helps with overfitting of the data.
# * **3rd Convolutional Layer ** -we add a Convoltuional Layer - extracts features from data source, these are kernels/filters and feature maps. Feature maps passed to the  next layer. This layer also has a ReLu activation function - Y = max(0, x) this removes any value <0 and prevents vanishing gradients or weights <0
# * **4th Pooling Layer ** - reduces dimensionality, reduce compute and helps with overfitting of the data.
# * **5th/6th Dense fully connected layer with softmax function:** put features together and classify what item of clothing is used

# > **Run some experiments to see how when you change the model below and rerun all the code the accuarcy and model will change:**
# * add a dropout layer after the first pooling layer and also before the final dense layer: `model.add(Dropout(0.5))`
# * change the value of dropout between 0 and 1: `model.add(Dropout(X))`
# * change the 2 Conv2D layer first variable to 32 instead of 64: `model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu'))`
# * Add padding to each of the Conv2D layers: `model.add(Conv2D(32, kernel_size=(3,3), padding = 'same', activation = 'relu'))`

run.log("Creating model:",0)
#Define the CNN model
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

#model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

run.log("Creating model:",1)

# * This code compiles the CNN model and assigns loss/optimiser functions as well as metrics we wish to view
# * I start a timer so we know how long the model takes to run
# * Fit the training data to the model using 24 epoches and batches of 64 images. Pass in the test data as your validation set so we can see how the accuracy differs on the training set to the validation set as the model runs through 24 epochs
# * Finally evaluate the model using the test/validation set
# 
# > Have a look at what optimisers are available in Keras and see what happens when you change this value: [https://keras.io/optimizers/](https://keras.io/optimizers/)


#compile - how to measure loss
model.compile(loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


#train the model and return loss and accuracy for each epoch - history dictionary
start = time.time()
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = time.time()

#evaluate the model on the test data
score = model.evaluate(x_test, y_test, verbose=0)

# log a single value
run.log("Final test loss", score[0])
print('Test Loss: ', score[0])

run.log('Final test accuracy', score[1])
print('Test Accuracy: ', score[1])
print('Time to run: ', (end-start))

score = model.evaluate(x_test, y_test, verbose=0)



# * This code plots the training and validation accuracy across 24 epochs
# * The training accuracy is often higher but the validation accuracy is deemed a more real world value

epoch_list = list(range(1, len(hist.history['accuracy']) + 1))
plt.plot(epoch_list, hist.history['accuracy'], epoch_list, hist.history['val_accuracy'])
plt.legend(('Training Accuracy', "Validation Accuracy"))
plt.show()

# log an image
#run.log_image('Accuracy vs Loss', plot=plt)

# Margaret Maynard Reid shows us a great way to visualise a sample set output of the test results here: [https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a](https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a)
# 
# Run this code to see a set of 15 images from the test set and whether the labels are assigned correctly

predictions = model.predict(x_test)

# Plot a random sample of 10 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(predictions[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))


    
run.log_image('Predictions', plot=figure)

os.makedirs('./outputs/model', exist_ok=True)

# serialize NN architecture to JSON
model_json = model.to_json()
# save model JSON
with open('./outputs/model/model.json', 'w') as f:
    f.write(model_json)
# save model weights
model.save_weights('./outputs/model/model.h5')
print("model saved in ./outputs/model folder")


Overwriting ./mnist-fashion/train.py


In [35]:
### Copying helper python files to script folder

In [131]:
import shutil
#shutil.copy('train.py', script_folder)
shutil.copy('utils.py', script_folder)

'./mnist-fashion/utils.py'

In [132]:
!ls -la *.py

-rw-r--r-- 1 nbuser nbuser 912 Mar  5 20:13 utils.py


In [122]:
#from azureml.train.estimator import Estimator
#
#script_params = {
#    '--data-folder': ds.as_mount()
#}

#print('source_directory:',script_folder)
#est = Estimator(source_directory=script_folder,
#                script_params=script_params,
#                compute_target=compute_target,
#                entry_script='train.py',
#                conda_packages=['tensorflow==2.0','keras', 'scikit-learn'])

In [133]:
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0']

In [134]:
print(ds.as_mount())

$AZUREML_DATAREFERENCE_workspaceblobstore


In [135]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ds.as_mount(),#dataset.as_named_input('mnist').as_mount(),
    '--batch-size': 64,
    '--epochs':4,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.001
}

est = TensorFlow(source_directory=script_folder,
                 entry_script='train.py',
                 script_params=script_params,
                 compute_target=compute_target,
                 pip_packages=['keras', 'matplotlib'],
                 framework_version='2.0',
                 use_gpu=False)

In [136]:
run = exp.submit(config=est)

In [137]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [138]:
run.wait_for_completion(show_output=True) # specify True for a verbose log

RunId: deeplearning_mnist_fashion_1583694920_cdf895f8
Web View: https://ml.azure.com/experiments/deeplearning_mnist_fashion/runs/deeplearning_mnist_fashion_1583694920_cdf895f8?wsid=/subscriptions/95975e47-f719-45de-9813-fea7d69635c0/resourcegroups/rg-ml-simple/workspaces/my-ml

Streaming azureml-logs/55_azureml-execution-tvmps_c37414b64b7d0bbe24ec9d6a25e12329a734c9d49fe60381d366e65401a692cb_d.txt

2020-03-08T19:16:02Z Starting output-watcher...
2020-03-08T19:16:02Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_4d1077f67abd45be421586dc6826c641
Digest: sha256:c1c624753f067a2d3c65b4a49ad9a77a8e162ed2d4bb74290d7c8d36d15801d8
Status: Image is up to date for mymlc6be2210.azurecr.io/azureml/azureml_4d1077f67abd45be421586dc6826c641:latest
e77ab00f7c0889f000b0a50bcc97519d3ed4e98b99f4dfb96439e6e4914fbf4b
2020/03/08 19:16:04 Version: 3.0.01154.0001 Branch: master Commit: fd92aa9d
2020/03/08 19:16:06 /de

Train on 60000 samples, validate on 10000 samples
Epoch 1/4

 8320/60000 [===>..........................] - ETA: 53s - loss: 0.8752 - accuracy: 0.6868


17216/60000 [=======>......................] - ETA: 43s - loss: 0.6946 - accuracy: 0.7504


23232/60000 [==========>...................] - ETA: 37s - loss: 0.6264 - accuracy: 0.7743


29248/60000 [=============>................] - ETA: 31s - loss: 0.5886 - accuracy: 0.7877


...

44672/60000 [=====================>........] - ETA: 15s - loss: 0.2428 - accuracy: 0.9094


50752/60000 [========================>.....] - ETA: 9s - loss: 0.2412 - accuracy: 0.9099


56704/60000 [===========================>..] - ETA: 3s - loss: 0.2413 - accuracy: 0.9099


60000/60000 [==============================] - 63s 1ms/step - loss: 0.2414 - accuracy: 0.9097 - val_loss: 0.2769 - val_accuracy: 0.9005
Test Loss:  0.2768705739736557
Test Accuracy:  0.9004999995231628
Time to run:  252.97141003608704
model saved in ./outputs/model folder


The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 300.0 seconds
2 items cleaning up...
Cleanup took 0.2662162780761719 seconds
Starting the daemon thread to refresh tokens in background for process with pid = 130

Streaming azureml-logs/75_job_post-tvmps_c37414b64b7d0bbe24ec9d6a25e12329a734c9d49fe60381d366e65401a692cb_d.txt

bash: /azureml-envs/azureml_607ee7b65c0b39097daefd395d9b3eef/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2020-03-08T19:21:10.970473
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 451
Jo

{'runId': 'deeplearning_mnist_fashion_1583694920_cdf895f8',
 'target': 'cpumlcluster',
 'status': 'Completed',
 'startTimeUtc': '2020-03-08T19:16:05.955329Z',
 'endTimeUtc': '2020-03-08T19:21:34.661184Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'd70af925-51bb-4b07-8772-6cc7f0475070',
  'AzureML.DerivedImageName': 'azureml/azureml_4d1077f67abd45be421586dc6826c641',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_workspaceblobstore',
   '--batch-size',
   '64',
   '--epochs',
   '4',
   '--first-layer-neurons',
   '300',
   '--second-layer-neurons',
   '100',
   '--learning-rate',
   '0.001'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpumlcluster',
  'dataReferences': {'wor

In [139]:
print(run.get_metrics())

{'Number of classes': 10, 'Sample image': 'aml://artifactId/ExperimentRun/dcid.deeplearning_mnist_fashion_1583694920_cdf895f8/Sample image_1583695001.png', 'Creating model:': [0, 1], 'Final test loss': 0.2768705739736557, 'Final test accuracy': 0.9004999995231628, 'Predictions': 'aml://artifactId/ExperimentRun/dcid.deeplearning_mnist_fashion_1583694920_cdf895f8/Predictions_1583695263.png'}


In [106]:
print(run.get_file_names())

['Accuracy vs Loss_1583693324.png', 'Sample image_1583693071.png', 'azureml-logs/55_azureml-execution-tvmps_c37414b64b7d0bbe24ec9d6a25e12329a734c9d49fe60381d366e65401a692cb_d.txt', 'azureml-logs/65_job_prep-tvmps_c37414b64b7d0bbe24ec9d6a25e12329a734c9d49fe60381d366e65401a692cb_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_c37414b64b7d0bbe24ec9d6a25e12329a734c9d49fe60381d366e65401a692cb_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/128_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model/model.h5', 'outputs/model/model.json']


### Registering the Model with the Workspace
Register the model to use in your workspace. 

In [140]:
model = run.register_model(model_name='keras-cnn-mnist-fashion', model_path='outputs/model')

In [141]:
print(model.name, model.id, model.version, sep='\t')

keras-cnn-mnist-fashion	keras-cnn-mnist-fashion:3	3


In [142]:
# Create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [164]:
# Optionally, delete the Azure Machine Learning Compute cluster
#compute_target.delete()

## Deploy models on Azure ML

Now we are ready to deploy the model as a web service running on your [local](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#local) machine, in Azure Container Instance [ACI](https://azure.microsoft.com/en-us/services/container-instances/) or Azure Kubernetes Service [AKS](https://azure.microsoft.com/en-us/services/kubernetes-service/). Azure Machine Learning accomplishes this by constructing a Docker image with the scoring logic and model baked in. 
> **Note:** For this Notebook, we'll use the original model format for deployment, but the ONNX model can be deployed in the same way by using ONNX Runtime in the scoring script.

![](./images/aml-deploy.png)


### Deploying a web service
Once you've tested the model and are satisfied with the results, deploy the model as a web service. For this Notebook, we'll use the original model format for deployment, but note that the ONNX model can be deployed in the same way by using ONNX Runtime in the scoring script.

To build the correct environment, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the web service
* The model you trained before

Read more about deployment [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where)

### Create score.py

First, we will create a scoring script that will be invoked by the web service call. We have prepared a [score.py script](code/scoring/score.py) in advance that scores your BERT model.

* Note that the scoring script must have two required functions, ``init()`` and ``run(input_data)``.
    * In ``init()`` function, you typically load the model into a global object. This function is executed only once when the Docker container is started.
    * In ``run(input_data)`` function, the model is used to predict a value based on the input data. The input and output to run typically use JSON as serialization and de-serialization format but you are not limited to that.

In [162]:
%%writefile score.py
# scoring script used by service to load model and generate prediction
import json
import numpy as np
import os
from keras.models import load_model
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('shape-classifier-keras')
    model = load_model(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data - the image(s) to be classified.
    data = np.array(json.loads(raw_data)['data'])
    
    # Pre-process the images
    imgfeatures = data.astype('float32')
    imgfeatures /= 255

    # Get a prediction from the model
    predictions = model.predict(imgfeatures)
    # get thge classname for the highest probability prediction for each input
    classnames = ['circle', 'square', 'triangle']
    predicted_classes = []
    for prediction in predictions:
        class_idx = np.argmax(prediction)
        predicted_classes.append(classnames[int(class_idx)])
    # Return the predictions as a JSON
    return json.dumps(predicted_classes)

Overwriting score.py


In [163]:
%pycat score.py

### Create Environment

You can create and/or use a Conda environment using the [Conda Dependencies object](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) when deploying a Webservice.

In [158]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pandas'],
                                 pip_packages=['numpy','pandas','keras','inference-schema[numpy-support]','azureml-defaults','tensorflow==2.0.0','transformers==2.0.0'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [159]:
%pycat myenv.yml

## Create Inference Configuration

We need to define the [Inference Configuration](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) for the web service. There is support for a source directory, you can upload an entire folder from your local machine as dependencies for the Webservice.
Note: in that case, your entry_script and conda_file paths are relative paths to the source_directory path.

Sample code for using a source directory:

```python
inference_config = InferenceConfig(source_directory="C:/abc",
                                   runtime= "python", 
                                   entry_script="x/y/score.py",
                                   conda_file="env/myenv.yml")
```

 - source_directory = holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - runtime = Which runtime to use for the image. Current supported runtimes are 'spark-py' and 'python
 - entry_script = contains logic specific to initializing your model and running predictions
 - conda_file = manages conda and python package dependencies.
 
 
 > **Note:** Deployment uses the inference configuration deployment configuration to deploy the models. The deployment process is similar regardless of the compute target. Deploying to AKS is slightly different because you must provide a reference to the AKS cluster.

In [160]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory="./",
                                   runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="myenv.yml"
                                  )

In [161]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import LocalWebservice

# Create a local deployment for the web service endpoint
deployment_config = LocalWebservice.deploy_configuration()
# Deploy the service
local_service = Model.deploy(
    ws, "mymodel", [model], inference_config, deployment_config)
# Wait for the deployment to complete
local_service.wait_for_deployment(True)
# Display the port that the web service is available on
print(local_service.port)

TypeError: load_config() got an unexpected keyword argument 'config_dict'